# Testing the DEB model
## Bremer and van Kessel (1990)
1. Importing libraries

In [1]:
import numpy as np
import pandas as pd
from scipy.integrate import odeint
from scipy.optimize import dual_annealing
from scipy.optimize import differential_evolution

2. Defining DEB model

In [2]:
def DEBmodel (y, t, pars):
    #define initial pools (eu is assumed to be zero)
    Sl=y[0];    el=y[1];    X1l=y[2];     CO2l=y[3];
    X1u=y[4]
    #define parameters
    yA=1; 
    Km=pars[0];     
    v=pars[1];
    m=pars[2]; 
    g=pars[3]; 
    ce=pars[4];
    MX1=ce/4;
    #Scaling function for substrate uptake
    f=Sl/(Km+Sl) #labelled substrate only
    
    #Isotope signals
    eatm = 1
    X1atm = X1l/(X1l + X1u)
    
    #Fluxes
    uptake=(v*ce/yA)*(X1l+X1u)*f #labelled substrate only
    growth = (v*el-m*g)/(el + g)
    
    #Define derivatives
    ##Labelled pools
    dSldt = -uptake
    deldt = v*(f - el)
    dX1ldt = max(0, (X1l + X1u)*growth) + min(0, (X1l + X1u)*growth*X1atm) 
    dCO2ldt = uptake*(1 - yA) + ce*((X1l + X1u)*el*(v-growth)) - max(0, (X1l + X1u)*growth)*MX1 - min(0, (X1l + X1u)*growth*X1atm)*MX1 
    ##Unabelled pools
    #dSudt
    #deudt = - v*eu
    dX1udt = min(0, (X1l + X1u)*growth*(1-X1atm)) 
        
    return dSldt, deldt, dX1ldt, dCO2ldt, dX1udt;

3. Defining outputs from DEB model

In [3]:
def calcDEB (model, pars, t, y0):
    #model parameters
    ##Km, v, m, g, ce
    pars_model=pars[0:5]
    #conversion factors
    ##ce, nX1
    conversions=pars[4:6]
    
    #solve the model
    y=odeint(model,y0,t, args=(pars_model,))
    #calculate labelled biomass (Bl), kec factor, and labelled chloroform flush (Flush14C) 
    Bl=0.25*conversions[0]*y[:, 2] + conversions[0]*y[:, 1]*(y[:, 2] + y[:, 4])
    kec = (0.25*conversions[1] + y[:, 1])/(0.25 + y[:, 1])
                
    #Create data with predictions
    yhat = np.concatenate((y[:, 0].reshape(len(d.Time),1), #Glucose
                           y[:, 3].reshape(len(d.Time),1),#CO2
                           kec.reshape(len(d.Time),1),
                           Bl.reshape(len(d.Time),1)), axis=1)
    
    return yhat

4. Defining objective function

In [4]:
def obj_funDEB (x):
    #define parameters
    ##Km, v, m, g, ce, nX1
    pars = x
    #initial conditions
    Sl_i = d.Sinit[0]
    # el_i = 0, Xl_i = 0, CO2l = 0, CO2u = 0, eu = 0  
    X1u_i = d.Cmicinit[0]/(pars[4]/4)
    
    y0 = np.array([Sl_i, 0, 0, 0,X1u_i])
    #times
    t = d.Time
    #model simulations
    yhat_full = calcDEB(DEBmodel, pars, t, y0)
    #observations
    obs=np.concatenate((np.array([d.S]).reshape(len(d.Time),1),
                        np.array([d.CO214cumul]).reshape(len(d.Time),1),
                        np.array([d.kec]).reshape(len(d.Time),1),
                        np.array([d.Cmic14]).reshape(len(d.Time),1)), axis=1)
    #weights
    weights=np.concatenate((np.nanstd(d.S).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanstd(d.CO214cumul).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanstd(d.kec).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanstd((d.Cmic14)).repeat(len(d.Time)).reshape(len(d.Time),1)),
                       axis=1)
    out=np.nansum(((yhat_full-obs)/weights)**2)
    return out

5. Calculate the goodness of fit

In [5]:
def goodnessDEB (x):
    #define parameters
    ##Km, v, m, g, ce, nX1
    pars = x
    #initial conditions
    Sl_i = d.Sinit[0]
    # el_i = 0, Xl_i = 0, CO2l = 0, CO2u = 0, eu = 0  
    X1u_i = d.Cmicinit[0]/(pars[4]/4)
    
    y0 = np.array([Sl_i, 0, 0, 0,X1u_i])
    #times
    t = d.Time
    #model simulations
    yhat_full = calcDEB(DEBmodel, pars, t, y0)
    
    #observations
    obs=np.concatenate((np.array([d.S]).reshape(len(d.Time),1),
                        np.array([d.CO214cumul]).reshape(len(d.Time),1),
                        np.array([d.kec]).reshape(len(d.Time),1),
                        np.array([d.Cmic14]).reshape(len(d.Time),1)), axis=1)
    
    R2=1-np.nansum((obs-yhat_full)**2)/np.nansum((obs-np.nanmean(obs))**2)
    ll=-np.nansum((obs-yhat_full)**2)/2/np.nanstd(obs)**2
    AIC = len(pars)*2 - 2*ll
    out = np.array([R2, ll, AIC])
    return out

Following function return the model solution for visualization in R

In [6]:
def predDEB (model, pars, t, y0):
    #model parameters
    ##Km, v, m, g, ce
    pars_model=pars[0:5]
    #conversion factors
    ##ce, nX1
    conversions=pars[4:6]
    
    #solve the model
    y=odeint(model,y0,t, args=(pars_model,))
    #calculate labelled biomass (Bl), kec factor, and labelled chloroform flush (Flush14C) 
    Bl=0.25*conversions[0]*y[:, 2] + conversions[0]*y[:, 1]*(y[:, 2] + y[:, 4])
    kec = (conversions[1]/4 + y[:, 1])/(0.25 + y[:, 1])
        
    #Create data with predictions
    yhat = np.concatenate((y[:, 0].reshape(len(np.arange(0, 7.1, 0.1)),1), #Glucose
                           y[:, 3].reshape(len(np.arange(0, 7.1, 0.1)),1),#CO2
                           kec.reshape(len(np.arange(0, 7.1, 0.1)),1),
                           Bl.reshape(len(np.arange(0, 7.1, 0.1)),1)), axis=1)
    
    return yhat

## High glucose/high N
Reading data

In [7]:
dAll = pd.read_csv('/mnt/580CBE2464C5F83D/pracovni/data_statistika/SoilMBVariability/SoilMBVariabilityData/BremerKessel1990.csv', sep=',')
d = dAll[(dAll.Treatment=="HCHN")]
d = d.reset_index()
print(d)

   index                        Study        Soil Substrate  Clay   pH  Ctot  \
0     12  Bremer and van Kessel, 1990  silty loam   Glucose   NaN  7.4   1.3   
1     13  Bremer and van Kessel, 1990  silty loam   Glucose   NaN  7.4   1.3   
2     14  Bremer and van Kessel, 1990  silty loam   Glucose   NaN  7.4   1.3   
3     15  Bremer and van Kessel, 1990  silty loam   Glucose   NaN  7.4   1.3   
4     16  Bremer and van Kessel, 1990  silty loam   Glucose   NaN  7.4   1.3   
5     17  Bremer and van Kessel, 1990  silty loam   Glucose   NaN  7.4   1.3   

   Ntot  Temperature  Time   Cmicinit      Sinit     Cmic14  CO214cumul  \
0  0.17         22.5  0.00  11.606994  24.979184   0.000000    0.000000   
1  0.17         22.5  0.25  11.606994  24.979184   9.682721    1.206915   
2  0.17         22.5  0.50  11.606994  24.979184  19.286559    4.056026   
3  0.17         22.5  1.00  11.606994  24.979184  17.452561    7.182134   
4  0.17         22.5  3.00  11.606994  24.979184  15.973531    8

Estimating parameters

In [8]:
BremerKessel1990DA=dual_annealing(obj_funDEB, [#(0.05, 1), #yA
                                              (10, 1000), #Km
                                              (0.001, 20), #v
                                              (1e-12, 0.1), #m
                                              (0.1, 3), #g
                                              (0.1, 10), #ce
                                              (0, 1)]) #nX1
                                              #(0, 1) #ne
                                              #(0, 1)]) #eu_i

In [9]:
print(BremerKessel1990DA)
print(goodnessDEB(BremerKessel1990DA.x))

     fun: 0.7586540988343939
 message: ['Maximum number of iteration reached']
    nfev: 19897
    nhev: 0
     nit: 1000
    njev: 1128
  status: 0
 success: True
       x: array([1.61327938e+02, 9.06465065e+00, 2.91107062e-02, 3.34620228e-01,
       4.10471952e+00, 3.49716337e-01])
[ 0.98352259 -0.18949024 12.37898047]


In [10]:
BremerKessel1990DE=differential_evolution(obj_funDEB, [#(0.05, 1), #yA
                                              (10, 1000), #Km
                                              (0.001, 20), #v
                                              (1e-12, 0.1), #m
                                              (0.1, 3), #g
                                              (0.1, 10), #ce
                                              (0, 1)]) #nX1
                                              #(0, 1) #ne
                                              #(0, 1)]) #eu_i

In [11]:
print(BremerKessel1990DE)
print(goodnessDEB(BremerKessel1990DE.x))

     fun: 0.7587379608090821
     jac: array([ 0.04426203,  0.00667149, -0.03117709,  0.06655928, -0.02030304,
       -0.00389975])
 message: 'Optimization terminated successfully.'
    nfev: 3403
     nit: 33
 success: True
       x: array([1.62245466e+02, 9.09459349e+00, 2.91527881e-02, 3.34537333e-01,
       8.54335866e+00, 3.49880577e-01])
[ 0.98345159 -0.19030676 12.38061351]


In [12]:
np.savetxt('/mnt/580CBE2464C5F83D/pracovni/data_statistika/SoilMBVariability/PythonScripts/BremerHCHNPars.csv', BremerKessel1990DA.x.reshape(1,6), delimiter=",")

Exporting solution for R

In [13]:
#initial conditions
Sl_i = d.Sinit[0]
X1u_i = d.Cmicinit[0]/(BremerKessel1990DA.x[4]/4)
    
y0 = np.array([Sl_i, 0, 0, 0,X1u_i])

#times
t = np.arange(0, 7.1, 0.1)
#model simulations
HCHNPred = predDEB(DEBmodel, BremerKessel1990DA.x, t, y0)
#print(HCHNPred)

## High glucose
Reading data

In [14]:
d = dAll[(dAll.Treatment=="HC")]
d = d.reset_index()
print(d)

   index                        Study        Soil Substrate  Clay   pH  Ctot  \
0      6  Bremer and van Kessel, 1990  silty loam   Glucose   NaN  7.4   1.3   
1      7  Bremer and van Kessel, 1990  silty loam   Glucose   NaN  7.4   1.3   
2      8  Bremer and van Kessel, 1990  silty loam   Glucose   NaN  7.4   1.3   
3      9  Bremer and van Kessel, 1990  silty loam   Glucose   NaN  7.4   1.3   
4     10  Bremer and van Kessel, 1990  silty loam   Glucose   NaN  7.4   1.3   
5     11  Bremer and van Kessel, 1990  silty loam   Glucose   NaN  7.4   1.3   

   Ntot  Temperature  Time   Cmicinit      Sinit     Cmic14  CO214cumul  \
0  0.17         22.5  0.00  11.606994  24.979184   0.000000    0.000000   
1  0.17         22.5  0.25  11.606994  24.979184   9.367194    1.226701   
2  0.17         22.5  0.50  11.606994  24.979184  16.525702    3.957098   
3  0.17         22.5  1.00  11.606994  24.979184  17.077874    7.716342   
4  0.17         22.5  3.00  11.606994  24.979184  15.717166    9

Estimating parameters

In [15]:
BremerKessel1990DA=dual_annealing(obj_funDEB, [#(0.05, 1), #yA
                                              (10, 1000), #Km
                                              (0.001, 20), #v
                                              (1e-12, 0.1), #m
                                              (0.1, 3), #g
                                              (0.1, 10), #ce
                                              (0, 1)]) #nX1
                                              #(0, 1) #ne
                                              #(0, 1)]) #eu_i

In [16]:
print(BremerKessel1990DA)
print(goodnessDEB(BremerKessel1990DA.x))

     fun: 1.0018744489573128
 message: ['Maximum number of iteration reached']
    nfev: 23222
    nhev: 0
     nit: 1000
    njev: 1603
  status: 0
 success: True
       x: array([4.74955672e+02, 1.93727269e+01, 3.84267866e-02, 3.31058110e-01,
       8.80693035e+00, 3.41001002e-01])
[ 0.98981443 -0.11713408 12.23426815]


In [17]:
BremerKessel1990DE=differential_evolution(obj_funDEB, [#(0.05, 1), #yA
                                              (10, 1000), #Km
                                              (0.001, 20), #v
                                              (1e-12, 0.1), #m
                                              (0.1, 3), #g
                                              (0.1, 10), #ce
                                              (0, 1)]) #nX1
                                              #(0, 1) #ne
                                              #(0, 1)]) #eu_i

In [18]:
print(BremerKessel1990DE)
print(goodnessDEB(BremerKessel1990DE.x))

     fun: 1.0019986147991926
     jac: array([ 0.01433619,  0.00154052, -0.12637835,  0.00069333,  0.01574716,
        0.00038394])
 message: 'Optimization terminated successfully.'
    nfev: 2855
     nit: 28
 success: True
       x: array([4.77700441e+02, 1.94334127e+01, 3.84546154e-02, 3.30995877e-01,
       7.39489884e+00, 3.41082001e-01])
[ 0.98965903 -0.11892117 12.23784234]


In [19]:
np.savetxt('/mnt/580CBE2464C5F83D/pracovni/data_statistika/SoilMBVariability/PythonScripts/BremerHCPars.csv', BremerKessel1990DA.x.reshape(1,6), delimiter=",")

In [20]:
#initial conditions
Sl_i = d.Sinit[0]
X1u_i = d.Cmicinit[0]/(BremerKessel1990DA.x[4]/4)
    
y0 = np.array([Sl_i, 0, 0, 0,X1u_i])

#times
t = np.arange(0, 7.1, 0.1)
#model simulations
HCPred = predDEB(DEBmodel, BremerKessel1990DA.x, t, y0)

## Low glucose
Reading data

In [21]:
d = dAll[(dAll.Treatment=="LC")]
d = d.reset_index()
print(d)

   index                        Study        Soil Substrate  Clay   pH  Ctot  \
0      0  Bremer and van Kessel, 1990  silty loam   Glucose   NaN  7.4   1.3   
1      1  Bremer and van Kessel, 1990  silty loam   Glucose   NaN  7.4   1.3   
2      2  Bremer and van Kessel, 1990  silty loam   Glucose   NaN  7.4   1.3   
3      3  Bremer and van Kessel, 1990  silty loam   Glucose   NaN  7.4   1.3   
4      4  Bremer and van Kessel, 1990  silty loam   Glucose   NaN  7.4   1.3   
5      5  Bremer and van Kessel, 1990  silty loam   Glucose   NaN  7.4   1.3   

   Ntot  Temperature  Time   Cmicinit     Sinit    Cmic14  CO214cumul  \
0  0.17         22.5  0.00  11.606994  2.497918  0.000000    0.000000   
1  0.17         22.5  0.25  11.606994  2.497918  2.196853    0.276997   
2  0.17         22.5  0.50  11.606994  2.497918  2.088391    0.379881   
3  0.17         22.5  1.00  11.606994  2.497918  2.035146    0.441216   
4  0.17         22.5  3.00  11.606994  2.497918  1.932600    0.550037   
5

Estimating parameters

In [22]:
BremerKessel1990DA=dual_annealing(obj_funDEB, [#(0.05, 1), #yA
                                              (10, 1000), #Km
                                              (0.001, 20), #v
                                              (1e-12, 0.1), #m
                                              (0.1, 3), #g
                                              (0.1, 10), #ce
                                              (0, 1)]) #nX1
                                              #(0, 1) #ne
                                              #(0, 1)]) #eu_i

In [23]:
print(BremerKessel1990DA)
print(goodnessDEB(BremerKessel1990DA.x))

     fun: 5.35361250193482
 message: ['Maximum number of iteration reached']
    nfev: 13996
    nhev: 0
     nit: 1000
    njev: 285
  status: 0
 success: True
       x: array([10.00000115, 19.99999999,  0.02179999,  0.28135824,  5.13542865,
        0.44813304])
[ 0.99822572 -0.02040419 12.04080838]


In [24]:
BremerKessel1990DE=differential_evolution(obj_funDEB, [#(0.05, 1), #yA
                                              (10, 1000), #Km
                                              (0.001, 20), #v
                                              (1e-12, 0.1), #m
                                              (0.1, 3), #g
                                              (0.1, 10), #ce
                                              (0, 1)]) #nX1
                                              #(0, 1) #ne
                                              #(0, 1)]) #eu_i

In [25]:
print(BremerKessel1990DE)
print(goodnessDEB(BremerKessel1990DE.x))

     fun: 5.354321439335611
     jac: array([ 4.59543476e-04,  1.60875744e-03,  1.69304570e-03,  1.47659662e-03,
       -1.59872117e-05, -5.13011855e-04])
 message: 'Optimization terminated successfully.'
    nfev: 3594
     nit: 38
 success: True
       x: array([11.31528879, 19.94958378,  0.02178823,  0.28132485,  5.79899672,
        0.44812405])
[ 0.99823765 -0.02026699 12.04053398]


In [26]:
np.savetxt('/mnt/580CBE2464C5F83D/pracovni/data_statistika/SoilMBVariability/PythonScripts/BremerLCPars.csv', BremerKessel1990DE.x.reshape(1,6), delimiter=",")

In [27]:
#initial conditions
Sl_i = d.Sinit[0]
X1u_i = d.Cmicinit[0]/(BremerKessel1990DE.x[4]/4)
    
y0 = np.array([Sl_i, 0, 0, 0,X1u_i])

#times
t = np.arange(0, 7.1, 0.1)
#model simulations
LCPred = predDEB(DEBmodel, BremerKessel1990DE.x, t, y0)

Merging and exporting solutions

In [28]:
BremerKessel1990Pred = np.concatenate((LCPred, HCPred, HCHNPred))
print(BremerKessel1990Pred)
np.savetxt('/mnt/580CBE2464C5F83D/pracovni/data_statistika/SoilMBVariability/PythonScripts/BremerKessel1990Pred.csv', BremerKessel1990Pred, delimiter=",")

[[ 2.49791840e+00  0.00000000e+00  4.48124055e-01  0.00000000e+00]
 [ 4.43591774e-04  2.48071490e-01  4.68644515e-01  2.24940327e+00]
 [ 3.08720616e-08  3.27060455e-01  4.51010980e-01  2.17085782e+00]
 [-6.69771380e-11  3.62339329e-01  4.48518516e-01  2.13557901e+00]
 [ 7.69918412e-12  3.74287720e-01  4.48177743e-01  2.12363062e+00]
 [ 1.83717894e-12  3.79901159e-01  4.48131358e-01  2.11801718e+00]
 [-1.02731424e-11  3.84645497e-01  4.48125048e-01  2.11327284e+00]
 [-1.78114518e-09  3.89263197e-01  4.48124190e-01  2.10865514e+00]
 [ 3.97123899e-11  3.93855057e-01  4.48124073e-01  2.10406328e+00]
 [ 5.01906005e-09  3.98434784e-01  4.48124057e-01  2.09948355e+00]
 [-1.22961989e-10  4.03004257e-01  4.48124055e-01  2.09491408e+00]
 [-8.33964494e-11  4.07563742e-01  4.48124055e-01  2.09035459e+00]
 [ 5.02261574e-12  4.12113298e-01  4.48124055e-01  2.08580504e+00]
 [ 3.75189820e-12  4.16652956e-01  4.48124055e-01  2.08126538e+00]
 [-5.99442444e-13  4.21182731e-01  4.48124055e-01  2.07673560e